In [140]:
# Dependencies and Setup
import pandas as pd
import os
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy
# File to Load (Remember to Change These)
file_to_load2 = "DataSets/cleaned_billboard_lyrics.csv"

# Read Purchasing File and store into Pandas data frame
billboard_lyrics_data = pd.read_csv(file_to_load2, encoding = "ISO-8859-1")

#Get an idea for what the data looks like 
billboard_lyrics_data.head(5)

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [57]:
# rds_connection_string = "postgres:Ucb@2019@localhost:5432/RythmAnalytics"
# engine = create_engine(f'postgresql://{rds_connection_string}')
# Base = automap_base()
# # reflect the tables
# Base.prepare(engine, reflect=True)

# # Save reference to the table

# session = Session(engine)
# engine.table_names()
# billboard_lyrics_data.to_sql(name='Rolling_Stones', con=engine, if_exists='replace', index=False)
# pd.read_sql_query('SELECT * FROM "Rolling_Stones"', con=engine).head()

In [141]:
# import necessary libraries
import os
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
#################################################
# Flask Setup
#################################################
app = Flask(__name__)

#################################################
# Database Setup
#################################################

from flask_sqlalchemy import SQLAlchemy
rds_connection_string = "db/RythmAnalytics.sqlite"
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///db/RythmAnalytics.sqlite"
engine = create_engine(f'sqlite:///{rds_connection_string}')
# app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL', '')
db = SQLAlchemy(app)

# from .models import Pet
billboard_lyrics_data.to_sql(name="db_lyrics", con=engine,if_exists = 'replace',index=False)
pd.read_sql_query('SELECT * FROM "db_lyrics"', con=engine)

C:\Users\aga641\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0
5,6,downtown,petula clark,1965,when youre alone and life is making you lonel...,1.0
6,7,help,the beatles,1965,help i need somebody help not just anybody hel...,3.0
7,8,cant you hear my heart beat,hermans hermits,1965,carterlewis every time i see you lookin my way...,5.0
8,9,crying in the chapel,elvis presley,1965,you saw me crying in the chapel the tears i s...,1.0
9,10,my girl,the temptations,1965,ive got sunshine on a cloudy day when its cold...,3.0


In [246]:
# billboard_lyrics_data = billboard_lyrics_data.head(5)
def word_count(str):
    counts = dict()
    words = str.split(" ")
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

def getCountWordLyrics(billboard_lyrics_data):
    billboard_lyrics_data = billboard_lyrics_data[billboard_lyrics_data["Rank"].isin([1,2])]
    wordlist = []
    countlist = []
    yearList =[]
    sightWordsList = ['of','your','','you','a','the','i','na','it','to','if','that','had','into','on','is','its','and','she','so','x','yeah','im','in','this','p','up','let','me','my']
    for row in billboard_lyrics_data.itertuples():
        music_lyrics = billboard_lyrics_data["Lyrics"][row.Index]
        year = billboard_lyrics_data["Year"][row.Index]
        count = word_count(str(music_lyrics))
        for k,v in count.items():
            if k not in sightWordsList:
                wordlist.append(k)
                countlist.append(v)
                yearList.append(year)
    df= pd.DataFrame({"Year":yearList,"Word":wordlist,"Count":countlist})
    df = df[(df["Count"]>15)]
    df= df.sort_values(by=["Year","Count"],ascending =False)
    df.head()
    return df

getCountWordLyrics(billboard_lyrics_data)

,Year,Word,Count
9201,2015,funk,31
9200,2015,uptown,29
9209,2015,believe,18
9210,2015,just,18
9211,2015,watch,18
9177,2015,hot,16
8939,2014,happy,28
8940,2014,clap,24
8941,2014,along,24
8933,2014,like,19
